In [1]:
!nvidia-smi

Tue Nov 12 21:05:53 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2070 ...    Off |   00000000:01:00.0  On |                  N/A |
| N/A   84C    P0             65W /   80W |     971MiB /   8192MiB |     19%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
#%pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12
#%pip install cucim-cu12 cupy-cuda12x

In [3]:
#%load_ext cudf.pandas
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

#from cucim.skimage.exposure import rescale_intensity
import tensorflow as tf
#import cupy as cp
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split

print("Is torch using cuda? ",torch.cuda.is_available())
print("Is tensorflow using cuda? ",tf.test.is_built_with_cuda())
print("Is pandas using cuda? ",pd)


2024-11-12 21:05:53.685249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 21:05:53.706561: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-12 21:05:53.712705: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 21:05:53.729125: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Is torch using cuda?  True
Is tensorflow using cuda?  True
Is pandas using cuda?  <module 'pandas' from '/usr/local/lib/python3.11/dist-packages/pandas/__init__.py'>


In [4]:
name_mapping = [
    "box",
    "circularTorus",
    "cone",
    "coneOffset",
    "cylinder",
    "cylinderSlope",
    "dish",
    "mesh",
    "pyramid",
    "rectangularTorus",
    "sphere"
]

In [5]:
def sort_by_number(texts:list[str]):
    def key(text:str):
        text = re.sub(r'.*photos_', '', text)
        text = re.sub(r'\.csv', '', text)
        text = re.sub(r'\D', '', text)
        return int(text)
    return sorted(texts, key=key)

In [9]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import glob
import gc

base_path = '/home/workspace/geometry-classifier/data/'
parquet_files = sorted(glob.glob(base_path + 'photos_v3_parquet/*.parquet'))

def data_generator():
    counter = 0
    encoder = LabelEncoder()
    
    for file in parquet_files:
        df = pd.read_parquet(file)
        
        # Drop unnecessary columns and encode labels
        df = df.drop(columns=['id'], axis=1)
        df['name'] = encoder.fit_transform(df['name'])
        
        # Split data into train, validation, and test sets
        X_train, X_aux, y_train, y_aux = train_test_split(df.drop(columns=["name"]), df['name'], test_size=0.4, random_state=42)
        X_val, X_test, y_val, y_test = train_test_split(X_aux, y_aux, test_size=0.5, random_state=42)
        
        # Ensure consistency between X and Y sizes
        if len(X_train) != len(y_train) or len(X_val) != len(y_val):
            raise ValueError("Mismatch in number of samples between features and labels")
        
        # Save training data every 5 iterations
        if counter % 5 == 0:
            training_file = f"training_{counter // 5}.parquet"
            training_df = pd.DataFrame(X_train)
            training_df['name'] = y_train
            training_df.to_parquet(base_path + f"training/{training_file}")
        
        # Efficiently reshape each row to (224, 224, 1)
        X = np.array([x.reshape(224, 224, 1).astype(np.uint8) for x in X_train.to_numpy()])
        X_val = np.array([x.reshape(224, 224, 1).astype(np.uint8) for x in X_val.to_numpy()])
        
        # Ensure labels are numpy arrays
        Y = y_train.values
        y_val = y_val.values

        yield X, Y, X_val, y_val
        
        # Cleanup to free memory
        del df, X_train, y_train, X_test, y_test, Y, X, X_val, y_val, X_aux, y_aux
        gc.collect()
        
        counter += 1


In [7]:
# 224 x 224

from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint

model = models.Sequential([
    # First Convolutional Block with reduced filters
    layers.Conv2D(8, (3, 3), activation='relu', input_shape=(224, 224, 1)),
    layers.MaxPooling2D((2, 2)),
    
    # Second Convolutional Block with reduced filters
    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Third Convolutional Block with reduced filters
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Global Average Pooling instead of Flatten
    layers.GlobalAveragePooling2D(),
    
    # Final Dense Layers
    layers.Dense(32, activation='relu'),  # Reduced dense layer size
    layers.Dropout(0.4),  # Dropout for regularization
    layers.Dense(10, activation='softmax')  # Output layer for 10 classes
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1731445558.760387     445 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731445558.768062     445 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1731445558.768297     445 cuda_executor.cc:

In [10]:
checkpoint_callback = ModelCheckpoint('model.keras',save_best_only=True, save_weights_only=False, mode='min', verbose=1)

epochs = 100

for i in range(100):
    print(f"Epochs {i+1}/{epochs}")
    for X_train, Y_train, X_val, Y_val in data_generator():
        cnn = model.fit(X_train,Y_train, epochs=1, steps_per_epoch=len(parquet_files), callbacks=[checkpoint_callback], batch_size=8, validation_data=(X_val, Y_val), verbose=1)



Epochs 1/100


I0000 00:00:1731445830.015846     512 service.cc:146] XLA service 0x770798005900 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731445830.015888     512 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 2070 with Max-Q Design, Compute Capability 7.5
2024-11-12 21:10:30.075586: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-12 21:10:30.355581: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906


  35/1042 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 1.0000 - loss: 2.2873

I0000 00:00:1731445833.685589     512 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


  77/1042 ━━━━━━━━━━━━━━━━━━━━ 38s 40ms/step - accuracy: 1.0000 - loss: 2.2685

...

  70/1042 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 1.0000 - loss: 2.0025
Epoch 1: val_loss improved from 2.03179 to 1.90209, saving model to model.keras
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 1s 464us/step - accuracy: 1.0000 - loss: 1.9700 - val_accuracy: 1.0000 - val_loss: 1.9021


2024-11-12 21:11:37.434991: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 11082665919636872685
2024-11-12 21:11:37.435038: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 13971001384966430482
/usr/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


  66/1042 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 1.0000 - loss: 1.8753
Epoch 1: val_loss improved from 1.90209 to 1.77672, saving model to model.keras
1042/1042 ━━━━━━━━━━━━━━━━━━━━ 1s 507us/step - accuracy: 1.0000 - loss: 1.8423 - val_accuracy: 1.0000 - val_loss: 1.7767


2024-11-12 21:12:06.017105: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_4]]
2024-11-12 21:12:06.017151: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 11082665919636872685
/usr/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-11-12 21:12:06.017174: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 13971001384966430482


KeyboardInterrupt: 